# 📊 Data Structures & Algorithms Performance Analysis

This notebook provides comprehensive performance analysis and visualization of our enhanced algorithms.

## 🎯 Objectives
- Benchmark algorithm performance
- Visualize time complexity characteristics
- Compare implementations
- Demonstrate real-world usage patterns

In [ ]:
# Import required libraries
import sys
import os
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from typing import List, Tuple
import tempfile
from pathlib import Path

# Add src directory to path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

from enhanced_lru_cache import LRUCache
from enhanced_file_finder import FileSearcher

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 🔄 LRU Cache Performance Analysis

Let's analyze the performance characteristics of our LRU Cache implementation.

In [ ]:
def benchmark_lru_cache() -> pd.DataFrame:
    """Benchmark LRU Cache operations across different sizes."""
    capacities = [10, 50, 100, 500, 1000, 5000]
    operations_count = 1000
    results = []
    
    for capacity in capacities:
        cache = LRUCache(capacity)
        
        # Benchmark SET operations
        start_time = time.perf_counter()
        for i in range(operations_count):
            cache.set(i, f"value_{i}")
        set_time = time.perf_counter() - start_time
        
        # Benchmark GET operations
        start_time = time.perf_counter()
        for i in range(operations_count):
            cache.get(random.randint(0, operations_count - 1))
        get_time = time.perf_counter() - start_time
        
        results.append({
            'capacity': capacity,
            'set_time_ms': set_time * 1000,
            'get_time_ms': get_time * 1000,
            'set_ops_per_sec': operations_count / set_time,
            'get_ops_per_sec': operations_count / get_time
        })
        
        print(f"Capacity {capacity:4d}: SET {set_time*1000:6.2f}ms, GET {get_time*1000:6.2f}ms")
    
    return pd.DataFrame(results)

# Run benchmark
lru_results = benchmark_lru_cache()
lru_results.head()

In [ ]:
# Visualize LRU Cache Performance
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Time vs Capacity
ax1.plot(lru_results['capacity'], lru_results['set_time_ms'], 'o-', label='SET operations', linewidth=2, markersize=8)
ax1.plot(lru_results['capacity'], lru_results['get_time_ms'], 's-', label='GET operations', linewidth=2, markersize=8)
ax1.set_xlabel('Cache Capacity')
ax1.set_ylabel('Time (milliseconds)')
ax1.set_title('LRU Cache: Time vs Capacity')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Operations per second
ax2.bar(lru_results['capacity'], lru_results['set_ops_per_sec'], alpha=0.7, label='SET ops/sec')
ax2.bar(lru_results['capacity'], lru_results['get_ops_per_sec'], alpha=0.7, label='GET ops/sec')
ax2.set_xlabel('Cache Capacity')
ax2.set_ylabel('Operations per Second')
ax2.set_title('LRU Cache: Throughput Analysis')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Memory usage simulation
memory_usage = lru_results['capacity'] * 64  # Assume 64 bytes per entry
ax3.plot(lru_results['capacity'], memory_usage, 'ro-', linewidth=2, markersize=8)
ax3.set_xlabel('Cache Capacity')
ax3.set_ylabel('Memory Usage (bytes)')
ax3.set_title('LRU Cache: Memory Usage')
ax3.grid(True, alpha=0.3)

# Efficiency comparison
efficiency = lru_results['get_ops_per_sec'] / lru_results['capacity']
ax4.plot(lru_results['capacity'], efficiency, 'go-', linewidth=2, markersize=8)
ax4.set_xlabel('Cache Capacity')
ax4.set_ylabel('Efficiency (ops/sec per capacity unit)')
ax4.set_title('LRU Cache: Efficiency Analysis')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 LRU Cache Performance Summary:")
print(f"• Average SET time: {lru_results['set_time_ms'].mean():.2f}ms")
print(f"• Average GET time: {lru_results['get_time_ms'].mean():.2f}ms")
print(f"• Peak throughput: {lru_results['get_ops_per_sec'].max():.0f} ops/sec")
print(f"• Efficiency scales: {'linearly' if efficiency.std() < efficiency.mean() * 0.1 else 'non-linearly'}")

## 🔍 File Search Performance Analysis

Analyzing the performance of our enhanced file search implementation.

In [ ]:
def create_test_file_structure(base_dir: str, num_dirs: int, files_per_dir: int) -> Tuple[str, int]:
    """Create a test directory structure with specified parameters."""
    total_files = 0
    
    for i in range(num_dirs):
        dir_path = os.path.join(base_dir, f"dir_{i}")
        os.makedirs(dir_path, exist_ok=True)
        
        for j in range(files_per_dir):
            # Create mix of file types
            extensions = ['.txt', '.py', '.c', '.md', '.json']
            ext = extensions[j % len(extensions)]
            file_path = os.path.join(dir_path, f"file_{j}{ext}")
            Path(file_path).touch()
            total_files += 1
    
    return base_dir, total_files

def benchmark_file_search() -> pd.DataFrame:
    """Benchmark file search across different directory sizes."""
    test_configs = [
        (10, 10),    # 100 files
        (20, 15),    # 300 files
        (30, 20),    # 600 files
        (50, 20),    # 1000 files
        (100, 25),   # 2500 files
    ]
    
    results = []
    searcher = FileSearcher()
    
    for num_dirs, files_per_dir in test_configs:
        with tempfile.TemporaryDirectory() as temp_dir:
            # Create test structure
            _, total_files = create_test_file_structure(temp_dir, num_dirs, files_per_dir)
            
            # Benchmark single extension search
            start_time = time.perf_counter()
            txt_files = searcher.find_files('.txt', temp_dir)
            single_search_time = time.perf_counter() - start_time
            
            # Benchmark multiple extension search
            start_time = time.perf_counter()
            multi_results = searcher.find_multiple_extensions(
                {'.txt', '.py', '.c'}, temp_dir
            )
            multi_search_time = time.perf_counter() - start_time
            
            results.append({
                'total_files': total_files,
                'directories': num_dirs,
                'files_per_dir': files_per_dir,
                'single_search_ms': single_search_time * 1000,
                'multi_search_ms': multi_search_time * 1000,
                'found_files': len(txt_files),
                'files_per_ms': total_files / (single_search_time * 1000)
            })
            
            print(f"Files: {total_files:4d}, Single: {single_search_time*1000:6.2f}ms, Multi: {multi_search_time*1000:6.2f}ms")
    
    return pd.DataFrame(results)

# Run file search benchmark
file_search_results = benchmark_file_search()
file_search_results.head()

In [ ]:
# Visualize File Search Performance
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Search time vs number of files
ax1.plot(file_search_results['total_files'], file_search_results['single_search_ms'], 
         'o-', label='Single Extension', linewidth=2, markersize=8)
ax1.plot(file_search_results['total_files'], file_search_results['multi_search_ms'], 
         's-', label='Multiple Extensions', linewidth=2, markersize=8)
ax1.set_xlabel('Total Files')
ax1.set_ylabel('Search Time (milliseconds)')
ax1.set_title('File Search: Time vs File Count')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Throughput analysis
ax2.bar(file_search_results['total_files'], file_search_results['files_per_ms'], alpha=0.7)
ax2.set_xlabel('Total Files')
ax2.set_ylabel('Files Processed per Millisecond')
ax2.set_title('File Search: Throughput Analysis')
ax2.grid(True, alpha=0.3)

# Linear complexity verification
# Fit linear trend line
z = np.polyfit(file_search_results['total_files'], file_search_results['single_search_ms'], 1)
p = np.poly1d(z)
ax3.plot(file_search_results['total_files'], file_search_results['single_search_ms'], 'ro', markersize=8, label='Actual')
ax3.plot(file_search_results['total_files'], p(file_search_results['total_files']), 'b--', linewidth=2, label='Linear Fit')
ax3.set_xlabel('Total Files')
ax3.set_ylabel('Search Time (milliseconds)')
ax3.set_title('File Search: Linear Complexity Verification')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Efficiency by directory structure
efficiency = file_search_results['total_files'] / file_search_results['single_search_ms']
ax4.scatter(file_search_results['directories'], efficiency, s=100, alpha=0.7)
ax4.set_xlabel('Number of Directories')
ax4.set_ylabel('Efficiency (files/ms)')
ax4.set_title('File Search: Efficiency vs Directory Count')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 File Search Performance Summary:")
print(f"• Average search time: {file_search_results['single_search_ms'].mean():.2f}ms")
print(f"• Peak throughput: {file_search_results['files_per_ms'].max():.1f} files/ms")
print(f"• Linear complexity: R² = {np.corrcoef(file_search_results['total_files'], file_search_results['single_search_ms'])[0,1]**2:.3f}")
print(f"• Multi-extension overhead: {(file_search_results['multi_search_ms'].mean() / file_search_results['single_search_ms'].mean() - 1) * 100:.1f}%")

## ⚡ Square Root Algorithm Analysis

Comparing different square root computation approaches.

In [ ]:
def sqrt_newton_method(x: int, precision: float = 1e-10) -> float:
    """Newton's method for square root (for comparison)."""
    if x < 0:
        raise ValueError("Cannot compute square root of negative number")
    if x == 0:
        return 0
    
    guess = x / 2
    while abs(guess * guess - x) > precision:
        guess = (guess + x / guess) / 2
    return guess

def sqrt_binary_search(x: int) -> int:
    """Binary search method (integer result)."""
    if x < 0:
        raise ValueError("Cannot compute square root of negative number")
    if x <= 1:
        return x
    
    left, right = 0, x
    while left <= right:
        mid = (left + right) // 2
        square = mid * mid
        
        if square == x:
            return mid
        elif square < x:
            if (mid + 1) * (mid + 1) > x:
                return mid
            left = mid + 1
        else:
            right = mid - 1
    return right

def benchmark_sqrt_algorithms() -> pd.DataFrame:
    """Benchmark different square root implementations."""
    test_numbers = [4, 16, 100, 1000, 10000, 100000, 1000000]
    iterations = 1000
    results = []
    
    for num in test_numbers:
        # Benchmark binary search
        start_time = time.perf_counter()
        for _ in range(iterations):
            sqrt_binary_search(num)
        binary_time = time.perf_counter() - start_time
        
        # Benchmark Newton's method
        start_time = time.perf_counter()
        for _ in range(iterations):
            sqrt_newton_method(num)
        newton_time = time.perf_counter() - start_time
        
        # Benchmark built-in
        start_time = time.perf_counter()
        for _ in range(iterations):
            int(num ** 0.5)
        builtin_time = time.perf_counter() - start_time
        
        results.append({
            'number': num,
            'binary_time_us': binary_time * 1000000 / iterations,
            'newton_time_us': newton_time * 1000000 / iterations,
            'builtin_time_us': builtin_time * 1000000 / iterations,
            'binary_result': sqrt_binary_search(num),
            'newton_result': int(sqrt_newton_method(num)),
            'builtin_result': int(num ** 0.5)
        })
        
        print(f"Number {num:7d}: Binary {binary_time*1000000/iterations:6.2f}μs, Newton {newton_time*1000000/iterations:6.2f}μs")
    
    return pd.DataFrame(results)

# Run sqrt benchmark
sqrt_results = benchmark_sqrt_algorithms()
sqrt_results.head()

In [ ]:
# Visualize Square Root Algorithm Performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Performance comparison
x_pos = np.arange(len(sqrt_results))
width = 0.25

ax1.bar(x_pos - width, sqrt_results['binary_time_us'], width, label='Binary Search', alpha=0.8)
ax1.bar(x_pos, sqrt_results['newton_time_us'], width, label="Newton's Method", alpha=0.8)
ax1.bar(x_pos + width, sqrt_results['builtin_time_us'], width, label='Built-in (**0.5)', alpha=0.8)

ax1.set_xlabel('Test Numbers')
ax1.set_ylabel('Time per Operation (microseconds)')
ax1.set_title('Square Root Algorithms: Performance Comparison')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([f'{int(x):,}' for x in sqrt_results['number']], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')  # Log scale for better comparison

# Algorithm complexity visualization
log_numbers = np.log10(sqrt_results['number'])
ax2.plot(log_numbers, np.log10(sqrt_results['binary_time_us']), 'o-', label='Binary Search', linewidth=2, markersize=8)
ax2.plot(log_numbers, np.log10(sqrt_results['newton_time_us']), 's-', label="Newton's Method", linewidth=2, markersize=8)
ax2.plot(log_numbers, np.log10(sqrt_results['builtin_time_us']), '^-', label='Built-in', linewidth=2, markersize=8)

ax2.set_xlabel('log₁₀(Input Number)')
ax2.set_ylabel('log₁₀(Time in microseconds)')
ax2.set_title('Square Root Algorithms: Complexity Analysis')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Square Root Algorithm Summary:")
print(f"• Binary Search avg time: {sqrt_results['binary_time_us'].mean():.2f}μs")
print(f"• Newton's Method avg time: {sqrt_results['newton_time_us'].mean():.2f}μs")
print(f"• Built-in avg time: {sqrt_results['builtin_time_us'].mean():.2f}μs")
print(f"• Fastest algorithm: {'Binary Search' if sqrt_results['binary_time_us'].mean() < sqrt_results['newton_time_us'].mean() else 'Newton Method'}")

# Verify correctness
correct_binary = (sqrt_results['binary_result'] == sqrt_results['builtin_result']).all()
correct_newton = (sqrt_results['newton_result'] == sqrt_results['builtin_result']).all()
print(f"• Binary search correctness: {'✅ Correct' if correct_binary else '❌ Incorrect'}")
print(f"• Newton method correctness: {'✅ Correct' if correct_newton else '❌ Incorrect'}")

## 🎯 Key Performance Insights

### LRU Cache
- **Time Complexity**: Consistent O(1) for both GET and SET operations
- **Space Complexity**: O(capacity) with minimal overhead
- **Scalability**: Linear memory usage, constant time operations

### File Search
- **Time Complexity**: O(n) where n is the number of files
- **Space Complexity**: O(d) where d is the maximum directory depth
- **Optimization**: Multiple extension search reduces redundant traversals

### Square Root Algorithms
- **Binary Search**: Consistent O(log n) time complexity
- **Newton's Method**: Quadratic convergence but more overhead
- **Built-in**: Highly optimized native implementation

## 🚀 Recommendations

1. **LRU Cache**: Excellent for high-frequency access patterns
2. **File Search**: Use multiple extension search for better efficiency
3. **Square Root**: Binary search optimal for integer results
4. **Memory Management**: All implementations use optimal space complexity